<a href="https://colab.research.google.com/github/drewamorbordelon/DS-Unit-2-Applied-Modeling/blob/master/module4-model-interpretation/10yr_regression_weekly_assetclasses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
%matplotlib inline
import sys

!pip install category_encoders==2.*

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='xgboost')

IndentationError: ignored

In [2]:
df = pd.read_csv('/content/weekly asset classes.csv', parse_dates=['date'], index_col='date')

In [3]:
df = df.iloc[14:]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1080 entries, 2000-01-09 to 2020-09-13
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gold            1080 non-null   float64
 1   rocgold         1080 non-null   float64
 2   goldbps         1080 non-null   float64
 3   copper          1080 non-null   float64
 4   roccop          1080 non-null   float64
 5   copbps          1080 non-null   float64
 6   wti             1080 non-null   float64
 7   rocwti          1080 non-null   float64
 8   wtibps          1080 non-null   float64
 9   dxy             1080 non-null   float64
 10  rocdxy          1080 non-null   float64
 11  dxybps          1080 non-null   float64
 12  eur             1080 non-null   float64
 13  roceur          1080 non-null   float64
 14  eurbps          1080 non-null   float64
 15  10yr            1080 non-null   float64
 16  roc10yr         1080 non-null   float64
 17  10yrbps        

In [5]:
# df['gold'] = df['gold'].str.strip(',').astype(float)

In [6]:
df.shape

(1080, 23)

In [7]:
df.head(1)

,gold,rocgold,goldbps,copper,roccop,copbps,wti,rocwti,wtibps,dxy,rocdxy,dxybps,eur,roceur,eurbps,10yr,roc10yr,10yrbps,regime,10yr2yr Spread,Agri,CRB,VIXCLS
date,,,,,,,,,,,,,,,,,,,,,,,
2000-01-09,284.0,0.070472,-89.709637,0.855,2.888087,-515.789037,28.02,4.786836,879.000703,101.72,-0.450186,39.577117,1.0128,0.456259,-115.298211,6.52,7.236842,269.875458,3.0,0.25,76.087,162.86,21.72


In [8]:
def wrangle(df):

  df = df.copy()

  # Remove the most extreme 1 % 
  df = df[(df['gold'] >= np.percentile(df['gold'], 0.5)) &
          (df['gold'] <= np.percentile(df['gold'], 99.5)) &
          (df['rocgold'] >= np.percentile(df['rocgold'], 0.5)) &
          (df['rocgold'] <= np.percentile(df['rocgold'], 99.5)) &
          (df['goldbps'] >= np.percentile(df['goldbps'], 0.5)) &
          (df['goldbps'] <= np.percentile(df['goldbps'], 99.5)) &

          (df['copper'] >= np.percentile(df['copper'], 0.5)) &
          (df['copper'] <= np.percentile(df['copper'], 99.5)) &
          (df['roccop'] >= np.percentile(df['roccop'], 0.5)) &
          (df['roccop'] <= np.percentile(df['roccop'], 99.5)) &
          (df['copbps'] >= np.percentile(df['copbps'], 0.5)) &
          (df['copbps'] <= np.percentile(df['copbps'], 99.5)) &

          (df['wti'] >= np.percentile(df['wti'], 0.5)) &
          (df['wti'] <= np.percentile(df['wti'], 99.5)) &
          (df['rocwti'] >= np.percentile(df['rocwti'], 0.5)) &
          (df['rocwti'] <= np.percentile(df['rocwti'], 99.5)) &
          (df['wtibps'] >= np.percentile(df['wtibps'], 0.5)) &
          (df['wtibps'] <= np.percentile(df['wtibps'], 99.5)) &

          (df['dxy'] >= np.percentile(df['dxy'], 0.5)) &
          (df['dxy'] <= np.percentile(df['dxy'], 99.5)) &
          (df['rocdxy'] >= np.percentile(df['rocdxy'], 0.5)) &
          (df['rocdxy'] <= np.percentile(df['rocdxy'], 99.5)) &
          (df['dxybps'] >= np.percentile(df['dxybps'], 0.5)) &
          (df['dxybps'] <= np.percentile(df['dxybps'], 99.5)) &

          (df['eur'] >= np.percentile(df['eur'], 0.5)) &
          (df['eur'] <= np.percentile(df['eur'], 99.5)) &
          (df['roceur'] >= np.percentile(df['roceur'], 0.5)) &
          (df['roceur'] <= np.percentile(df['roceur'], 99.5)) &
          (df['eurbps'] >= np.percentile(df['eurbps'], 0.5)) &
          (df['eurbps'] <= np.percentile(df['eurbps'], 99.5)) &

          (df['10yr'] >= np.percentile(df['10yr'], 0.5)) &
          (df['10yr'] <= np.percentile(df['10yr'], 99.5)) &
          (df['roc10yr'] >= np.percentile(df['roc10yr'], 0.5)) &
          (df['roc10yr'] <= np.percentile(df['roc10yr'], 99.5)) &
          (df['10yrbps'] >= np.percentile(df['10yrbps'], 0.5)) &
          (df['10yrbps'] <= np.percentile(df['10yrbps'], 99.5)) &

          (df['10yr2yr Spread'] >= np.percentile(df['10yr2yr Spread'], 0.5)) &
          (df['10yr2yr Spread'] <= np.percentile(df['10yr2yr Spread'], 99.5)) &
          (df['Agri'] >= np.percentile(df['Agri'], 0.5)) &
          (df['Agri'] <= np.percentile(df['Agri'], 99.5)) &
          (df['CRB'] >= np.percentile(df['CRB'], 0.5)) &
          (df['CRB'] <= np.percentile(df['CRB'], 99.5)) &
          (df['VIXCLS'] >= np.percentile(df['VIXCLS'], 0.5)) &
          (df['VIXCLS'] <= np.percentile(df['VIXCLS'], 99.5))]


  # Rename columns
  df = df.rename(columns={'gold': 'Gold', 
                          'rocgold': 'Gold ROC',
                          'goldbps': 'Gold bps',
                          'copper': 'Copper', 
                          'roccop': 'Copper ROC',
                          'copbps': 'Copper bps',
                          'wti': 'WTI',
                          'rocwti': 'WTI ROC',
                          'wtibps': 'WTI bps',
                          'dxy': 'DXY',
                          'rocdxy': 'DXY ROC',
                          'dxybps': 'DXY bps',
                          'eur': 'EUR',
                          'roceur': 'EUR ROC',
                          'eurbps': 'EUR bps',
                          '10yr': '10yr Rate',
                          'roc10yr': '10yr ROC',
                          '10yrbps': '10yr bps',
                          'regime': 'Regime'})
  
  # Turn Gold col into float instead of a string
  # df['Gold'] = df['Gold'].str.strip(',').astype(float)
  # df['Gold'] = pd.to_numeric(df['Gold'], downcast='float')
  # Rename column `regime`
  df['Regime'] = df['Regime'].replace(1.0, 'Regime 1')
  df['Regime'] = df['Regime'].replace(2.0, 'Regime 2')
  df['Regime'] = df['Regime'].replace(3.0, 'Regime 3')
  df['Regime'] = df['Regime'].replace(4.0, 'Regime 4')

  # Create CPI + or - column
  result = [] 
  for value in df['Regime']: 
    if value == 'Regime 1': 
      result.append("-") 
    elif value == 'Regime 2': 
      result.append('+') 
    elif value == 'Regime 3': 
      result.append('+') 
    else:
      result.append('-')

  df['CPI'] = result

        
  
  # Create GDP + or - column
  result1 = []
  for value in df['Regime']:
    if value == 'Regime 1': 
      result1.append('+') 
    elif value == 'Regime 2': 
      result1.append('+') 
    elif value == 'Regime 3': 
      result1.append('-') 
    else:
      result1.append('-')

  df['GDP'] = result1  

  # cols to poptentially drop `droppers`
  # droppers = ['Gold bps', 'Copper bps', 'WTI bps', 'DXY bps', 'EUR bps', 'Agri', 'CRB', 'Regime', '10yr Rate']  #'10yr bps'
  # droppers_alt = ['Gold ROC', 'Copper ROC', 'WTI ROC', 'DXY ROC', 'EUR ROC', '10yr ROC', 'Agri', 'CRB', 'Regime']
  
  # Keep only some columns
  # keep_cols = ['Gold',            #
              # 'Copper', 
              # 'WTI', 
              # 'DXY', 
              # 'EUR',
              # '10yr Rate',
              # 'Regime',
              # 'Gold bps']


  
  # df = df[keep_cols]
  # df = df.drop(columns=droppers_alt)
  # df['GDP Positive'] = (df['GDP'] == '+')
  df = df.drop(['10yr bps','10yr ROC', '10yr2yr Spread', 'EUR bps', 'Copper bps', 'Regime', 'Gold bps', 
                'Copper ROC', 'WTI ROC', 'WTI ROC', 'WTI bps', 'EUR', 'CPI'], axis=1)  #, 'Agri', 'CRB', 'Regime'
  # df = df[droppers]
  return df


In [9]:
df = wrangle(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 929 entries, 2000-02-20 to 2020-07-19
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Gold       929 non-null    float64
 1   Gold ROC   929 non-null    float64
 2   Copper     929 non-null    float64
 3   WTI        929 non-null    float64
 4   DXY        929 non-null    float64
 5   DXY ROC    929 non-null    float64
 6   DXY bps    929 non-null    float64
 7   EUR ROC    929 non-null    float64
 8   10yr Rate  929 non-null    float64
 9   Agri       929 non-null    float64
 10  CRB        929 non-null    float64
 11  VIXCLS     929 non-null    float64
 12  GDP        929 non-null    object 
dtypes: float64(12), object(1)
memory usage: 101.6+ KB


In [11]:
# mask = df['regime'] == 'Regime 3'
# df[mask]

In [12]:
# target is the regime column
# categorical
target = '10yr Rate'

y = df[target]
X = df.drop(target , axis=1)

In [13]:
cutoff1 = '2016-01-01'
cutoff2 = '2018-07-01'
# cutoff3 = '2020-01-01'
mask1 = (df.index <= cutoff1)
train = df[mask1]
val = df[~mask1]

mask1 = df.index < cutoff1
mask2 = (df.index >= cutoff1) & (df.index < cutoff2)
mask3 = (df.index >= cutoff2) 

val = df[mask2]
test = df[mask3]

In [14]:
y_train = train[target]
X_train = train.drop([target], axis=1)

y_val = val[target]
X_val = val.drop(target, axis=1)

y_test = test[target]
X_test = test.drop(target, axis=1)

In [15]:
# assert df.shape[0] == train.shape[0] + val.shape[0]
assert df.shape[0] == train.shape[0] + val.shape[0] + test.shape[0]

In [16]:
# Regression --> mean absolute error
from sklearn.metrics import mean_absolute_error
 
print('Baseline MAE:', mean_absolute_error(y_train, [y_train.mean()]*len(y_train)))
# print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

Baseline MAE: 1.0083524289066295


In [17]:
# Model 1: Linear Regression

model = make_pipeline(
    OrdinalEncoder(),   #OneHotEncoder(use_cat_names=True),
    # SimpleImputer(strategy='mean'),
    StandardScaler(),
    LinearRegression()  #LinearRegression()  RandomForestRegressor(random_state=42)
)

model.fit(X_train, y_train);

In [18]:
print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model.predict(X_test)))

Training MAE: 0.31806931369186686
Validation MAE: 0.24164468721532542
Test MAE: 0.5258533169990883


In [19]:
import numpy as np
coefficients = np.round(model.named_steps['linearregression'].coef_, 2)
list(zip(X_train.columns, coefficients))

[('Gold', -1.76),
 ('Gold ROC', 0.07),
 ('Copper', 0.87),
 ('WTI', 0.17),
 ('DXY', 0.29),
 ('DXY ROC', -0.11),
 ('DXY bps', -0.02),
 ('EUR ROC', -0.13),
 ('Agri', 0.21),
 ('CRB', -0.13),
 ('VIXCLS', 0.04),
 ('GDP', 0.09)]

In [20]:
# Model 2: Random Forest
model_f = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    RandomForestRegressor(n_jobs=-1, n_estimators=10)
)

model_f.fit(X_train, y_train);

In [21]:
print('Training MAE:', mean_absolute_error(y_train, model_f.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model_f.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model_f.predict(X_test)))

Training MAE: 0.05566063977746875
Validation MAE: 0.4425714285714285
Test MAE: 0.603956043956044


In [22]:
# from sklearn.metrics import classification_report, plot_confusion_matrix, plot_roc_curve

# # On validation data only
# print('Random Forest')
# print(classification_report(y_val, model.predict(X_val)))

In [23]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from xgboost import XGBClassifier, XGBRegressor

In [24]:
# Model 3: Gradient Boosting Regressor

model_grad = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    GradientBoostingRegressor(n_estimators=22)
)

model_grad.fit(X_train, y_train);

In [25]:
print('Training MAE:', mean_absolute_error(y_train, model_grad.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model_grad.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model_grad.predict(X_test)))

Training MAE: 0.2067200086779529
Validation MAE: 0.35189080176461907
Test MAE: 0.47654284675794


In [26]:
# Model 4: XGB Regressor

model_xgb = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBRegressor(n_jobs=-1, n_estimators=43)
)

model_xgb.fit(X_train, y_train);

[14:59:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [27]:
print('Training MAE:', mean_absolute_error(y_train, model_xgb.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model_xgb.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model_xgb.predict(X_test)))

Training MAE: 0.14635500267914836
Validation MAE: 0.32577631413435737
Test MAE: 0.45798841214441993


Feature Importances

In [31]:
model_xgb.named_steps['xgbclassifier'].feature_importances_

KeyError: ignored

In [ ]:
feature = 'Monthly Debts'

isolated = pdp_isolate(
    model=model_gb,
    dataset=X_val,
    model_features=X_val.columns,
    feature=feature
)

pdp_plot(isolated, feature_name=feature);

#PDP Plots

In [ ]:
# View Feature
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 72

In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_interact

In [ ]:
feature_imp = model.named_steps['xgbclassifier'].feature_importances_
features = X_train.columns

pd.Series(feature_imp, features).sort_values(ascending=False).head(7).plot(kind='barh');

In [ ]:
features = ['Copper', 'DXY']

interaction = pdp_interact(
    model=model_xgb,
    dataset=X_val,
    model_features=X_val.columns,
    features=features
)